In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")
train.head(10)

In [ ]:
import matplotlib.pyplot as plt

sex_pivot=train.pivot_table(index="Sex",values="Survived")
sex_pivot

In [ ]:
pclass_pivot=train.pivot_table(index="Pclass",values="Survived")
pclass_pivot.plot.bar()
plt.show()

In [ ]:
train['Age'].describe()

In [ ]:
train[train["Survived"]==1]

In [ ]:
survived=train[train["Survived"]==1]
died=train[train["Survived"]==0]
survived["Age"].plot.hist(alpha=0.5,color='red',bins=50)
died["Age"].plot.hist(alpha=0.5,color='blue',bins=50)
plt.legend(['Survived','Died'])
plt.show()

In [ ]:
def process_age(df,cut_points,label_names):
    df["Age"]=df["Age"].fillna(-0.5)
    df["Age_categories"]=pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points=[-1,0,5,12,18,35,60,100]
label_names=["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"]

train=process_age(train,cut_points,label_names)
test=process_age(test,cut_points,label_names)

age_cat_pivot=train.pivot_table(index='Age_categories',values="Survived")
age_cat_pivot.plot.bar()
plt.show()

In [ ]:
column_name="Pclass"
df=train
dummies=pd.get_dummies(df[column_name],prefix=column_name)
dummies.head()

In [ ]:
def create_dummies(df,column_name):
    dummies=pd.get_dummies(df[column_name],prefix=column_name)
    df=pd.concat([df,dummies],axis=1)
    return df

train=create_dummies(train,"Pclass")
test=create_dummies(test,"Pclass")
train.head()


In [ ]:
train=create_dummies(train,"Sex")
test=create_dummies(test,"Sex")
train=create_dummies(train,"Age_categories")
test=create_dummies(test,"Age_categories")
train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']
lr.fit(train[columns],train["Survived"])

In [ ]:
from sklearn.model_selection import train_test_split

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

all_X = train[columns]
all_y=train['Survived']

train_X, test_X, train_y, test_y = train_test_split(all_X,all_y,test_size=0.2,random_state=0)
train_X.shape

In [ ]:
lr.fit(train_X,train_y)
predictions=lr.predict(test_X)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y,predictions)
accuracy

In [ ]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(lr,all_X,all_y,cv=10)
np.mean(scores)

In [ ]:
holdout=test
lr.fit(all_X,all_y)
holdout_predictions=lr.predict(holdout[columns])
holdout_predictions

In [ ]:
holdout_ids=holdout['PassengerId']
submission_df={"PassengerId":holdout_ids,"Survived":holdout_predictions}
submission=pd.DataFrame(submission_df)
submission.to_csv('titanic_submission.csv',index=False)